In [42]:
import pandas as pd
import numpy as np
import pygad
import random
import copy

In [43]:
import csv

# Initialize an empty dictionary
enginge_RUL = {}

with open('RUL_consultancy_predictions_A3-2.csv', newline='') as csvfile:
    # Create a CSV reader object with semicolon as delimiter
    reader = csv.reader(csvfile, delimiter=';')
    # Skip the header row
    next(reader)
    # Loop over each row in the CSV
    for row in reader:
        # Check if row has exactly 2 columns
        if len(row) == 2:
            enginge_RUL[int(row[1])] = int(row[0])

In [44]:
engine_RUL_30 = {key: value for key, value in enginge_RUL.items() if value <= 30}

In [45]:
Team_A_time = {key: 4 for key in range(1, 21)}
Team_A_time.update({key: 3 for key in range(21, 56)})
Team_A_time.update({key: 2 for key in range(56, 81)})
Team_A_time.update({key: 8 for key in range(81, 101)})

Team_B_time = {j : Team_A_time[j] + 1 for j in range(1, 26)}
Team_B_time.update({j : Team_A_time[j] + 2 for j in range(26, 71)})
Team_B_time.update({j : Team_A_time[j] + 1 for j in range(71, 101)})

In [46]:
T1_time = Team_A_time
T2_time = Team_B_time
T3_time = Team_A_time
T4_time = Team_B_time

# Random Initializer

In [47]:
def calculate_team_work_time(team, engines):
    if team in ["T1", "T3"]:
        engine_times = Team_A_time
    else:
        engine_times = Team_B_time
    
    time = 0
    
    for engine in engines:
        time += engine_times[engine]
    
    return time
    

def random_init(engines_RUL):
    schedule = {"T1":[], "T2":[], "T3":[], "T4":[]}
    
    possible_engines = list(engines_RUL.keys())
    possible_teams = ["T1", "T2", "T3", "T4"]
    
    iterations = 0 
    
    while possible_engines and iterations < 500:
        index = random.randrange(4)
        team = possible_teams[index]
        
        index = random.randrange(len(possible_engines))
        selected_engine = possible_engines[index]
        
        schedule[team].append(selected_engine)
        
        possible_engines.remove(selected_engine)
        
        iterations += 1

    return schedule

In [48]:
random_init(engine_RUL_30)

{'T1': [49, 56, 100, 90, 35, 76, 40],
 'T2': [91, 42, 81, 92],
 'T3': [37, 20, 68, 24, 34],
 'T4': [31, 61, 82, 36, 64, 66, 41, 77]}

# Loss Function

In [49]:
def engine_fix_date(schedule):
    engine_fix = {}
    for team in schedule:
        days = 0
        if team == "T1" or team == "T3":
            time_table = Team_A_time
        else:
            time_table = Team_B_time
            
        for engine in schedule[team]:
            engine_fix[engine] = days
            days = days + time_table[engine]
              
    return engine_fix


initial_schedule = random_init(engine_RUL_30)
print(initial_schedule)

engine_cost = {key: 4 for key in range(1, 21)}
engine_cost.update({key: 3 for key in range(21, 31)})
engine_cost.update({key: 2 for key in range(31, 46)})
engine_cost.update({key: 5 for key in range(46, 81)})
engine_cost.update({key: 6 for key in range(81, 100)})


def loss_function(schedule, engine_RUL, engine_cost):
    loss = 0
    fix_dates = engine_fix_date(schedule)
    print(fix_dates)
    print(engine_RUL)
    for team in schedule:
        for engine in schedule[team]:

            days_late = fix_dates[engine] - engine_RUL[engine]
            engine_loss = 0
            if days_late > 0:
                cost = engine_cost[engine]
    
                for i in range(days_late):
                    day_loss = cost * (i ** 2)
                    if day_loss > 250:
                        day_loss = 250
                    engine_loss += day_loss
            loss += engine_loss
        
    return loss  

loss_schedule = loss_function(initial_schedule, engine_RUL_30, engine_cost)
print(loss_schedule)

{'T1': [49, 31, 61, 35, 91, 42], 'T2': [34, 100, 64, 37, 41, 40], 'T3': [68, 82, 81, 20], 'T4': [92, 56, 66, 76, 90, 24, 36, 77]}
{49: 0, 31: 3, 61: 6, 35: 8, 91: 11, 42: 19, 34: 0, 100: 5, 64: 14, 37: 18, 41: 23, 40: 28, 68: 0, 82: 2, 81: 10, 20: 18, 92: 0, 56: 9, 66: 13, 76: 17, 90: 20, 24: 29, 36: 33, 77: 38}
{20: 10, 24: 26, 31: 14, 34: 8, 35: 8, 36: 24, 37: 21, 40: 29, 41: 23, 42: 13, 49: 14, 56: 18, 61: 23, 64: 24, 66: 18, 68: 13, 76: 3, 77: 27, 81: 6, 82: 11, 90: 28, 91: 29, 92: 24, 100: 24}
4827


# Termination Criterion

In [50]:
# TO DO LATER

# Selection of parents

In [51]:
# TO DO SELECT ONE DROM PYGAD UTILS
# https://pygad.readthedocs.io/en/latest/pygad.html#parent-selection-methods

# Crossover

In [181]:
parent_1 = random_init(engine_RUL_30)
parent_2 = random_init(engine_RUL_30)

In [182]:
# maybe TO DO: ONLY DELETE THE ORIGINAL DUPLICATES, NOT THE SWAPPED DUPLICATES

def double_fixer(child, parent):
    doubles = []
    skipped_ones = []
    numbers = {}
    
    for team in parent:
        for engine in parent[team]:
            numbers[engine] = [0]
    
    for team in child:
        index = -1
        for engine in child[team]:
            index = index + 1
            numbers[engine][0] = numbers[engine][0] + 1
            numbers[engine].append(team)
            numbers[engine].append(index)
                
    for key in numbers:
        if numbers[key][0] == 0:
            skipped_ones.append(key)
        elif numbers[key][0] > 1:
            doubles.append(key)
    
    for fault in doubles:
        team = numbers[fault][-2]
        index = numbers[fault][-1]        
        next_missed_one = skipped_ones.pop()
        child[team][index] = next_missed_one
        
    return child

def find_center(parent, team, middle_down_round):
    if len(parent[team]) % 2 == 0:
            middle_index = len(parent[team]) // 2
            next_middle_down_round = middle_down_round
    else:
        if middle_down_round:
            middle_index = len(parent[team]) // 2             
            next_middle_down_round = False
        else: 
            middle_index = (len(parent[team]) // 2) + 1
            next_middle_down_round = True
    
    return middle_index, next_middle_down_round


In [183]:
def crossover(parents, offspring_size = 2):
    parent_1 = parents[0]
    parent_2 = parents[1]
    
    child1 = {key: [] for key in parent_1}
    child2 = {key: [] for key in parent_2} 
    cross_side = random.choice(["Left", "Right"]) # Right means you keep the right side of p1 and switch the right side of p2
       
    p1_middle_down_round = True
    p2_middle_down_round = True
    
    for team in parent_1:
        
        p1_middle, p1_middle_down_round = find_center(parent_1, team, p1_middle_down_round)
        p2_middle, p2_middle_down_round = find_center(parent_2, team, p2_middle_down_round)

        
        if cross_side == "Left":
            sub_schedule_p1 = parent_1[team][:p1_middle]
            sub_schedule_p2 = parent_2[team][:p2_middle]
            
            child1[team].extend(sub_schedule_p2)
            child1[team].extend(parent_1[team][p1_middle:])
            
            child2[team].extend(sub_schedule_p1)
            child2[team].extend(parent_2[team][p2_middle:])
        
        if cross_side == "Right":
            sub_schedule_p1 = parent_1[team][p1_middle:]
            sub_schedule_p2 = parent_2[team][p2_middle:]
            
            child1[team].extend(parent_1[team][:p1_middle])
            child1[team].extend(sub_schedule_p2)
            
            child2[team].extend(parent_2[team][:p2_middle])
            child2[team].extend(sub_schedule_p1)
    
    
    child1 = double_fixer(child1, parent_1)
    child2 = double_fixer(child2, parent_2)
    
    return child1, child2


In [184]:
def print_parent_formatted(parent, name):
    formatted = ',\n  '.join(f"'{key}': {value}" for key, value in parent.items())
    print(f"{name} = {{\n  {formatted}\n}}")

In [338]:


print_parent_formatted(parent_1, "parent_1")
print_parent_formatted(parent_2, "parent_2")

child1, child2 = crossover(parents = [parent_1, parent_2])

print(child1)
print(child2)
print(sum(len(value) for value in child1.values()))
print(sum(len(value) for value in child2.values()))


parent_1 = {
  'T1': [64, 68, 92, 49, 66],
  'T2': [37, 91, 34, 76, 77, 90, 36],
  'T3': [40, 31, 42, 82, 24, 35, 81],
  'T4': [61, 41, 20, 56, 100]
}
parent_2 = {
  'T1': [40, 61, 81, 35, 100, 77],
  'T2': [82, 49],
  'T3': [64, 76, 20, 92, 68],
  'T4': [56, 41, 42, 66, 90, 24, 37, 34, 31, 91, 36]
}
{'T1': [64, 68, 35, 100, 77], 'T2': [37, 91, 34, 76, 49], 'T3': [40, 31, 42, 20, 92, 56], 'T4': [61, 41, 66, 81, 82, 90, 24, 36]}
{'T1': [40, 61, 81, 92, 49, 66], 'T2': [82, 77, 90, 36], 'T3': [64, 76, 31, 24, 35, 91], 'T4': [56, 41, 42, 37, 68, 20, 34, 100]}
24
24


In [19]:
test = {}
test.update({1 : 0})
test.update({2 : 0})
print(test[1]) 
test[1] = 4
test[1] = test[1] + 1
print(test)

0
{1: 5, 2: 0}


# Mutation

In [532]:
# This function chooses a random engine in the schema, and returns its index and the team it is in now
def random_chooser(child):
    total_length = 0
    team_lengths = {}
    for team in child:
        length = len(child[team])
        team_lengths[team] = length
        total_length = total_length + length
    
    index = random.randrange(total_length)
    for team in team_lengths:
        if index - team_lengths[team] >= 0:
            index = index - team_lengths[team]
        else:
            index_team = team
            break
            
    return [index, index_team]

In [534]:
# we kunnen swaps doen als mutaties
def swap_mutation(child):
    engine1  = random_chooser(child)
    engine2 = random_chooser(child)
    print(engine1)
    print(engine2)
    
    new_child = copy.deepcopy(child)
    
    new_child[engine1[1]][engine1[0]] = child[engine2[1]][engine2[0]]
    new_child[engine2[1]][engine2[0]] = child[engine1[1]][engine1[0]]
    
    return new_child

In [535]:
print(child1)
new_swap_child = swap_mutation(child1)
print(new_swap_child)

{'T1': [64, 68, 35, 100, 77], 'T2': [37, 91, 34, 76, 49], 'T3': [40, 31, 42, 20, 92, 56], 'T4': [61, 41, 66, 81, 82, 90, 24, 36]}
[1, 'T3']
[0, 'T1']
{'T1': [31, 68, 35, 100, 77], 'T2': [37, 91, 34, 76, 49], 'T3': [40, 64, 42, 20, 92, 56], 'T4': [61, 41, 66, 81, 82, 90, 24, 36]}


In [536]:
# alleen mis is het beter om additie te doen ipv swap, anders worden je teams nooit meer langer na eerste paar generaties, aangezien ik denk dat alles redleijk snel uit averaged, 
# en terwijl je dit soort variaties mis wel wil?

def additive_mutation(child):
    possible_teams = ["T1", "T2", "T3", "T4"]
    to_remove = random_chooser(child)
    child_copy = copy.deepcopy(child)
    
    engine = child_copy[to_remove[1]].pop(to_remove[0])
    team_index = random.randrange(4)
    team = possible_teams[team_index]
    length_team = len(child_copy[team])
    index = random.randrange(length_team)
    
    child_copy[team].insert(index, engine)
    return child_copy


In [537]:
print(child1)
new_additive_child = additive_mutation(child1)
print(new_additive_child)

{'T1': [64, 68, 35, 100, 77], 'T2': [37, 91, 34, 76, 49], 'T3': [40, 31, 42, 20, 92, 56], 'T4': [61, 41, 66, 81, 82, 90, 24, 36]}
{'T1': [64, 68, 35, 100, 77], 'T2': [37, 91, 34, 76, 49], 'T3': [40, 31, 42, 20, 92, 56], 'T4': [61, 41, 66, 24, 81, 82, 90, 36]}
